In [14]:
import pandas as pd
from sqlalchemy import create_engine
import os

In [15]:
from config.config_loader import Config

config = Config()

In [16]:
db_elec_path = config.get_resolved_path("data_path.elec.processed.facturas")
db_elec_manual_path = config.get_resolved_path("data_path.elec.processed.facturas_manual")
customers_path_single = config.get_resolved_path("data_path.elec.customers.folder")

In [17]:
from sqlalchemy import create_engine

engine = create_engine(f'sqlite:///{db_elec_path}', echo=True)

facturas_df_sql = pd.read_sql(('SELECT * FROM facturas'), con=engine);

facturas_df_manual = pd.read_pickle(db_elec_manual_path)

# if not facturas_df[facturas_df['cups'] == cups].empty:

facturas_df = pd.concat([facturas_df_sql, facturas_df_manual], ignore_index=True)
for column in ['inicio_periodo', 'fin_periodo', 'issue_date']:
    facturas_df[column] = pd.to_datetime(facturas_df[column])

2025-07-03 14:22:34,920 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-07-03 14:22:34,921 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("SELECT * FROM facturas")
2025-07-03 14:22:34,922 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-07-03 14:22:34,924 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("SELECT * FROM facturas")
2025-07-03 14:22:34,924 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-07-03 14:22:34,926 INFO sqlalchemy.engine.Engine SELECT * FROM facturas
2025-07-03 14:22:34,927 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-07-03 14:22:34,955 INFO sqlalchemy.engine.Engine ROLLBACK


In [18]:
facturas_df[facturas_df['alias'] == 'KEM ONE']

,num_factura,inicio_periodo,fin_periodo,issue_date,cups,coste_potencia,coste_energia,excesos_potencia,dto_electrointensivo,coste_reactiva,...,energia_p5,energia_p6,maximetros_p1,maximetros_p2,maximetros_p3,maximetros_p4,maximetros_p5,maximetros_p6,year,month
926,ABOELE_2300002110,2023-01-01,2023-01-31,2023-03-01,ES0021000003225511XB,-6338.05,-87395.69,0.00,0.0,0.0,...,0.0,-422513.0,0.0,0.0,0.0,0.0,0.0,0.0,2023,1
933,ABOELE_2300007160,2023-06-01,2023-06-30,2023-08-03,ES0021000003225511XB,-6133.61,-94081.53,0.00,0.0,0.0,...,0.0,-405595.0,0.0,0.0,0.0,0.0,0.0,0.0,2023,6
934,ABOELE_2300008114,2023-08-01,2023-08-31,2023-09-20,ES0021000003225511XB,0.00,-103245.51,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2023,8
953,ABOELE_2500005525,2025-04-01,2025-04-30,2025-05-13,ES0021000003225511XB,0.00,-237450.92,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2025,4
988,FELEC_2300032902,2022-11-01,2022-11-30,2023-01-26,ES0021000003225511XB,0.00,117236.06,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022,11
1002,FELEC_2300038121,2022-12-01,2022-12-31,2023-02-03,ES0021000003225511XB,0.00,124702.35,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022,12
1042,FELEC_2300071136,2023-01-01,2023-01-31,2023-02-22,ES0021000003225511XB,6338.05,87395.69,0.00,0.0,0.0,...,0.0,422513.0,0.0,0.0,0.0,0.0,0.0,0.0,2023,1
1059,FELEC_2300106401,2023-02-01,2023-02-28,2023-03-17,ES0021000003225511XB,0.00,109458.42,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2023,2
1127,FELEC_2300154146,2023-03-01,2023-03-31,2023-04-26,ES0021000003225511XB,0.00,77204.73,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2023,3
1163,FELEC_2300191771,2023-04-01,2023-04-30,2023-05-18,ES0021000003225511XB,0.00,66276.26,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2023,4


In [19]:
import invoice_comp.facturas as facturas
Facturas = facturas.FacturasElec(facturas_df)

In [20]:
facturas_df['cliente'].unique()

array(['PR', 'Catalonia Fundacio', 'Kem One', 'Grupo Inspired',
       'Sundolitt', 'Trilla y Famesa', 'Altan Pharma', 'Taghleef',
       'Montfrisa', 'VYOSA', 'PR2'], dtype=object)

In [21]:
# potencia.columns.get_loc('potencia_p1') -> saca el numero de la columna
Facturas.get_measurement('potencia');
Facturas.get_measurement('energia');
Facturas.get_measurement('maximetros');
Facturas.get_invoice_title();
Facturas.get_invoices();

In [22]:
Facturas.set_period('df_potencia');
Facturas.set_period('df_maximetros');
Facturas.set_period('df_energia');
Facturas.set_period('df_invoices');
Facturas.set_period('df_titles');

In [23]:
Facturas.update_column_names('df_potencia');
Facturas.update_column_names('df_maximetros');
Facturas.update_column_names('df_energia');
Facturas.update_column_names('df_titles')
Facturas.update_column_names('df_invoices')

In [24]:
Facturas.group_facturas()
Facturas.set_period('facturas')
df_facturas = Facturas.facturas
df_facturas

,cliente,alias,period,coste_potencia,coste_energia,excesos_potencia,dto_electrointensivo,coste_reactiva,total_bruto_ie_iva,total_bruto_iva,...,energia_p3,energia_p4,energia_p5,energia_p6,maximetros_p1,maximetros_p2,maximetros_p3,maximetros_p4,maximetros_p5,maximetros_p6
0,Altan Pharma,ALAVA 1,ene-25,1623.80,19028.38,0.00,0.0,0.0,20652.18,21708.06,...,0.0,0.0,0.0,61274.0,296.0,301.0,0.0,0.0,0.0,287.0
1,Altan Pharma,ALAVA 1,feb-25,1466.68,23458.94,0.00,0.0,0.0,24925.62,26199.99,...,0.0,0.0,0.0,64880.0,319.0,305.0,0.0,0.0,0.0,302.0
2,Altan Pharma,ALAVA 1,mar-25,1623.80,14438.29,0.00,0.0,0.0,16062.09,16883.30,...,34512.0,0.0,0.0,70599.0,0.0,302.0,306.0,0.0,0.0,308.0
3,Altan Pharma,ALAVA 1,abr-25,1571.44,7631.16,0.00,0.0,0.0,9202.60,9673.10,...,0.0,34869.0,27745.0,51441.0,0.0,0.0,0.0,287.0,309.0,303.0
4,Altan Pharma,ALAVA 1,may-25,1623.80,9780.10,7.98,0.0,0.0,11411.88,11995.33,...,0.0,43894.0,34043.0,61394.0,0.0,0.0,0.0,347.0,339.0,275.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025,VYOSA,CRISTALIA 8,ene-25,88.19,394.29,0.00,0.0,0.0,482.88,507.57,...,0.0,0.0,0.0,905.0,0.0,0.0,0.0,0.0,0.0,0.0
2030,VYOSA,CRISTALIA 9,oct-24,112.34,0.00,0.00,0.0,0.0,0.00,114.40,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2031,VYOSA,CRISTALIA 9,nov-24,108.75,0.00,0.00,0.0,0.0,0.00,110.75,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2032,VYOSA,CRISTALIA 9,dic-24,112.34,0.00,0.00,0.0,0.0,0.00,114.40,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [25]:
clientes_elec = os.listdir(customers_path_single)
clientes_elec_clean = []
for cliente in clientes_elec:
    if not '.' in cliente:
        clientes_elec_clean.append(cliente)
# clientes_elec_clean = ['Taghleef']
clientes_elec_clean

['Altan Pharma',
 'Catalonia Fundacio',
 'Generic',
 'Grupo Inspired',
 'Kem One',
 'Montfrisa',
 'PR',
 'PR2',
 'Sundolitt',
 'Taghleef',
 'Trilla y Famesa',
 'VYOSA']

In [26]:
import pandas as pd
from openpyxl import load_workbook
from openpyxl.utils.exceptions import InvalidFileException
from zipfile import BadZipFile

for cliente in clientes_elec_clean:

    excel_path = config.get_resolved_path('outputs.invoice_comp.elec.seguimiento', client_id=cliente)
    df_filtered = df_facturas[df_facturas['cliente'] == cliente]

    if not df_filtered.empty:
        print(f'Adding data for {cliente}\n')

        try:
            # Just to verify the file is a valid Excel file before writing
            load_workbook(excel_path)
            
            with pd.ExcelWriter(excel_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
                df_filtered.to_excel(writer, sheet_name='SQL', index=False)

        except (InvalidFileException, BadZipFile) as e:
            print(f"⚠️ Error: '{cliente}' - archivo dañado o inválido ({excel_path}). Se omite.\n")
            continue


Adding data for Altan Pharma

Adding data for Catalonia Fundacio

Adding data for Grupo Inspired

Adding data for Kem One

Adding data for Montfrisa

Adding data for PR

Adding data for PR2

Adding data for Sundolitt

Adding data for Taghleef



FileNotFoundError: Resolved path does not exist: c:\Users\mikel.perez\0_Python_projects_v2\outputs\invoice_comp\elec\Trilla y Famesa_Seguimiento Electricidad.xlsx

In [ ]:
df_filtered

,cliente,alias,period,coste_potencia,coste_energia,excesos_potencia,dto_electrointensivo,coste_reactiva,total_bruto_ie_iva,total_bruto_iva,...,energia_p3,energia_p4,energia_p5,energia_p6,maximetros_p1,maximetros_p2,maximetros_p3,maximetros_p4,maximetros_p5,maximetros_p6
1880,Taghleef,TAGHLEEF,ene-23,23167.49,238779.09,0.00,-36557.52,0.00,225390.20,226593.72,...,0.0,0.0,0.0,1176427.0,3984.0,3976.0,0.0,0.0,0.0,4112.0
1883,Taghleef,TAGHLEEF,feb-23,20925.47,316481.05,0.00,-32366.59,0.00,305041.01,306635.38,...,0.0,0.0,0.0,980333.0,3556.0,3516.0,0.0,0.0,0.0,3692.0
1886,Taghleef,TAGHLEEF,mar-23,23167.49,278856.58,0.00,-31857.50,0.00,270167.76,271595.17,...,490299.0,0.0,0.0,1136106.0,0.0,4076.0,4108.0,0.0,0.0,4072.0
1889,Taghleef,TAGHLEEF,abr-23,22420.15,218302.09,0.00,-21636.89,0.00,219086.50,220256.03,...,0.0,535804.0,407520.0,1177523.0,0.0,0.0,0.0,3820.0,3780.0,3724.0
1892,Taghleef,TAGHLEEF,may-23,23167.49,230776.48,0.00,-22708.76,0.00,2351720.00,232488.83,...,0.0,626264.0,484537.0,1240919.0,0.0,0.0,3220.0,10964.0,7968.0,10912.0
1895,Taghleef,TAGHLEEF,jun-23,20806.13,251580.06,0.00,-24205.28,156.53,249085.94,250405.47,...,604942.0,468049.0,0.0,1107397.0,0.0,0.0,10988.0,10972.0,0.0,10968.0
1897,Taghleef,TAGHLEEF,jul-23,22241.06,294760.35,5682.70,-32139.81,0.00,290545.49,292074.79,...,0.0,0.0,0.0,1337755.0,4468.0,4288.0,0.0,0.0,0.0,4192.0
1899,Taghleef,TAGHLEEF,ago-23,22241.06,189272.90,0.00,-18341.06,0.00,193174.09,194216.53,...,435569.0,330287.0,0.0,885139.0,0.0,0.0,3008.0,3000.0,0.0,3004.0
1901,Taghleef,TAGHLEEF,sep-23,21523.59,280541.81,0.00,-19201.84,0.00,282864.71,284353.13,...,601002.0,459304.0,0.0,1219382.0,0.0,0.0,3836.0,3900.0,0.0,3944.0
1903,Taghleef,TAGHLEEF,oct-23,22241.06,225749.74,0.00,-17059.88,0.00,230932.11,232163.34,...,0.0,505962.0,387082.0,1119503.0,0.0,0.0,0.0,3744.0,3816.0,3800.0


In [ ]:
###########
df_title_filtered = Facturas.df_titles[
    (Facturas.df_titles['cliente'] == 'Taghleef') & (Facturas.df_titles['ALIAS'] == 'TAGHLEEF')
        ].tail(1)
df_potencia_filtered = Facturas.df_potencia[
    (Facturas.df_potencia['cliente'] == 'Taghleef') & (Facturas.df_potencia['alias'] == 'TAGHLEEF')
        ].tail(1)
df_energia_filtered = Facturas.df_energia[
    (Facturas.df_energia['cliente'] == 'Taghleef') & (Facturas.df_energia['alias'] == 'TAGHLEEF')
    ].tail(12)
df_maximetros_filtered = Facturas.df_maximetros[
    (Facturas.df_maximetros['cliente'] == 'Taghleef') & (Facturas.df_maximetros['alias'] == 'TAGHLEEF')
    ].tail(12)
df_invoices_filtered = Facturas.df_invoices[
    (Facturas.df_invoices['cliente'] == 'Taghleef') & (Facturas.df_invoices['alias'] == 'TAGHLEEF')
    ].tail(12)

In [ ]:
df_title_filtered = df_title_filtered.drop('cliente', axis=1)

In [ ]:
df_potencia_filtered = df_potencia_filtered.drop(['cliente', 'alias', 'Periodo'], axis=1)

In [ ]:
df_energia_filtered = df_energia_filtered.drop(['cliente', 'alias'], axis=1)

In [ ]:
df_maximetros_filtered = df_maximetros_filtered.drop(['cliente', 'alias'], axis=1)

In [ ]:
print(df_title_filtered)
print(df_potencia_filtered)
(df_maximetros_filtered)

         ALIAS Periodo                  CUPS      Num. Factura Fecha emisión
1908  TAGHLEEF  may-25  ES0031101457946001KR  PI25142000204445    2025-06-14
          P1      P2      P3      P4      P5      P6
1894  4030.0  4030.0  4030.0  4030.0  4030.0  4830.0


,Periodo,P1,P2,P3,P4,P5,P6
1896,jun-24,0.0,0.0,3916.0,3860.0,0.0,3852.0
1898,jul-24,3952.0,3984.0,0.0,0.0,0.0,4064.0
1900,ago-24,0.0,0.0,3448.0,3424.0,0.0,3564.0
1902,sep-24,0.0,0.0,3916.0,3896.0,0.0,3868.0
1904,oct-24,0.0,0.0,0.0,4004.0,3972.0,3996.0
1906,nov-24,0.0,3988.0,3956.0,0.0,0.0,3924.0
1908,dic-24,3936.0,3860.0,0.0,0.0,0.0,3888.0
1882,ene-25,3740.0,3712.0,0.0,0.0,0.0,3948.0
1885,feb-25,4084.0,4052.0,0.0,0.0,0.0,4148.0
1888,mar-25,0.0,4180.0,4144.0,0.0,0.0,4216.0


In [ ]:
df_invoices_filtered = df_invoices_filtered.drop(['alias', 'cliente'], axis=1)
df_invoices_filtered

,Periodo,Coste Energía,Coste Reactiva,Coste Potencia,Coste excesos,Descuento electrointensivos,Base impuesto eléctrico,Base imponible
1896,jun-24,186690.77,0.0,20670.04,0.00,-19281.27,188079.73,197769.58
1898,jul-24,263436.56,0.0,21359.03,0.00,-31485.06,-982.05,266338.09
1900,ago-24,204001.16,0.0,21359.03,0.00,-18484.63,206875.75,217529.04
1902,sep-24,231538.36,0.0,20670.04,0.00,-19646.98,232561.61,244525.68
1904,oct-24,211827.79,0.0,21359.03,0.00,-17012.02,216174.99,227303.72
1906,nov-24,307528.40,0.0,20670.04,0.00,-21649.54,306549.09,322295.91
1908,dic-24,254411.34,0.0,21359.03,0.00,-24563.40,251207.16,264126.98
1882,ene-25,298608.92,0.0,20068.05,0.00,-20632.81,298044.56,313359.24
1885,feb-25,332606.19,0.0,18125.96,296.55,0.00,351029.06,369045.27
1888,mar-25,216947.13,0.0,20068.05,516.94,0.00,237532.52,249753.41


In [ ]:
print(df_maximetros_filtered.columns)
print(df_potencia_filtered.columns)

Index(['Periodo', 'P1', 'P2', 'P3', 'P4', 'P5', 'P6'], dtype='object')
Index(['P1', 'P2', 'P3', 'P4', 'P5', 'P6'], dtype='object')


In [ ]:

def highlight_rows(val):
    """Apply light grey background to cells with value 0."""
    if val == 0:
        color = 'background-color: white'
    else:
        color = 'background-color: #C2C7CA'
    return color

# Define the numeric columns
numeric_cols = df_energia_filtered.columns[1:]

styled_df = (df_energia_filtered.style
                .format(("{:,.3f}"), subset=numeric_cols)  # Format numeric columns as integers
                .set_table_styles([
                    {'selector': 'thead th',      
                     'props': [
                         ('background-color', '#012e41'),  
                         ('font-family', 'Montserrat, sans-serif'),
                         ('border', '1px solid grey'),
                         ('color', 'white'),               
                         ('text-align', 'center')          
                     ]},
                    {'selector': 'tbody td',      
                     'props': [
                         #('background-color', 'white'),#'#C2C7CA'), 
                         ('color', 'black'),              
                         ('font-family', 'Montserrat, sans-serif'),
                         ('text-align', 'center'),
                         ('min-width', '80px'),          
                     ]}
                ])
                .map(highlight_rows)
                .set_properties(**{
                    'border-collapse': 'collapse', 
                     'border': '1px solid grey' # Ensure no gaps between cells
                })
                .hide(axis='index')  # Hide index
             )
(styled_df)

Periodo,P1,P2,P3,P4,P5,P6,Total
jun-24,0.000,0.000,579.393,450.978,0.000,"1,314.860","2,345.231"
jul-24,691.560,536.354,0.000,0.000,0.000,"1,250.524","2,478.438"
ago-24,0.000,0.000,452.320,346.901,0.000,998.008,"1,797.229"
sep-24,0.000,0.000,621.853,479.940,0.000,"1,255.855","2,357.648"
oct-24,0.000,0.000,0.000,588.296,452.013,"1,103.794","2,144.103"
nov-24,0.000,556.147,430.295,0.000,0.000,"1,287.558","2,274.000"
dic-24,411.999,323.797,0.000,0.000,0.000,"1,022.312","1,758.108"
ene-25,539.271,415.101,0.000,0.000,0.000,"1,175.315","2,129.687"
feb-25,512.288,406.103,0.000,0.000,0.000,"1,148.999","2,067.390"
mar-25,0.000,573.128,449.942,0.000,0.000,"1,340.885","2,363.955"


In [ ]:
styled_df = (df_title_filtered.style
                # .format(("{:.0f} kW"), subset=numeric_cols)  # Format numeric columns as integers
                .set_table_styles([
                    {'selector': 'thead th',      
                     'props': [
                         ('background-color', '#012e41'),  
                         ('font-family', 'Montserrat, sans-serif'),
                         ('color', 'white'),               
                         ('border', '1px solid black'),
                         ('text-align', 'center')          
                     ]},
                    {'selector': 'tbody td',      
                     'props': [
                         ('background-color', '#012e41'), 
                         ('color', 'white'),              
                         ('font-family', 'Montserrat, sans-serif'),
                         ('border', '1px solid black'),
                         ('text-align', 'center')          
                     ]}
                ])
                #.map(highlight_zeros, subset=numeric_cols)
                .hide(axis='index')  # Apply highlight to zeros in numeric columns
             )

(styled_df)

ALIAS,Periodo,CUPS,Num. Factura,Fecha emisión
TAGHLEEF,may-25,ES0031101457946001KR,PI25142000204445,2025-06-14 00:00:00


In [ ]:
numeric_cols = df_invoices_filtered.columns[1:]
styled_df = (df_invoices_filtered.style
                .format("{:,.2f} €", subset=numeric_cols)  # Format numeric columns with two decimal places and euro symbol
                .set_table_styles([
                    {'selector': 'thead th',      
                     'props': [
                         ('background-color', '#012e41'),  
                         ('font-family', 'Montserrat, sans-serif'),
                         ('border', '1px solid grey'),
                         ('color', 'white'),               
                         ('text-align', 'center')          
                     ]},
                    {'selector': 'tbody td',      
                     'props': [
                         ('color', 'black'),              
                         ('font-family', 'Montserrat, sans-serif'),
                         ('text-align', 'center'),
                         ('min-width', '120px'),          
                     ]},
                    {'selector': 'tbody td:nth-child(1)',  # Specific to the first column
                     'props': [
                         ('min-width', '80px'),  # Adjust this value as needed to fit the content
                         ('white-space', 'nowrap'),  # Prevent wrapping in the first column
                         ('background-color', '#C2C7CA')
                     ]}
                ])
                .applymap(highlight_rows, subset=numeric_cols)  # Apply custom highlight function to numeric columns
                .set_properties(**{
                    'border-collapse': 'collapse', 
                    'border': '1px solid grey'  # Ensure no gaps between cells
                })
                .hide(axis='index')  # Hide index
             )

# Display the styled DataFrame
styled_df

                
(styled_df)

C:\Users\mikel.perez\AppData\Local\Temp\ipykernel_12220\3270065080.py:27: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  .applymap(highlight_rows, subset=numeric_cols)  # Apply custom highlight function to numeric columns


Periodo,Coste Energía,Coste Reactiva,Coste Potencia,Coste excesos,Descuento electrointensivos,Base impuesto eléctrico,Base imponible
jun-24,"186,690.77 €",0.00 €,"20,670.04 €",0.00 €,"-19,281.27 €","188,079.73 €","197,769.58 €"
jul-24,"263,436.56 €",0.00 €,"21,359.03 €",0.00 €,"-31,485.06 €",-982.05 €,"266,338.09 €"
ago-24,"204,001.16 €",0.00 €,"21,359.03 €",0.00 €,"-18,484.63 €","206,875.75 €","217,529.04 €"
sep-24,"231,538.36 €",0.00 €,"20,670.04 €",0.00 €,"-19,646.98 €","232,561.61 €","244,525.68 €"
oct-24,"211,827.79 €",0.00 €,"21,359.03 €",0.00 €,"-17,012.02 €","216,174.99 €","227,303.72 €"
nov-24,"307,528.40 €",0.00 €,"20,670.04 €",0.00 €,"-21,649.54 €","306,549.09 €","322,295.91 €"
dic-24,"254,411.34 €",0.00 €,"21,359.03 €",0.00 €,"-24,563.40 €","251,207.16 €","264,126.98 €"
ene-25,"298,608.92 €",0.00 €,"20,068.05 €",0.00 €,"-20,632.81 €","298,044.56 €","313,359.24 €"
feb-25,"332,606.19 €",0.00 €,"18,125.96 €",296.55 €,0.00 €,"351,029.06 €","369,045.27 €"
mar-25,"216,947.13 €",0.00 €,"20,068.05 €",516.94 €,0.00 €,"237,532.52 €","249,753.41 €"


In [ ]:
def highlight_rows(val):
    """Apply light grey background to cells with value 0."""
    if val == 0:
        color = 'background-color: white'
    else:
        color = 'background-color: #C2C7CA'
    return color

# Define the numeric columns
numeric_cols = df_potencia_filtered.columns[1:]

styled_df = (df_potencia_filtered.style
                .format(("{:.0f}"))  # Format numeric columns as integers
                .set_table_styles([
                    {'selector': 'thead th',      
                     'props': [
                         ('background-color', '#012e41'),  
                         ('font-family', 'Montserrat, sans-serif'),
                         ('border', '1px solid grey'),
                         ('color', 'white'),               
                         ('text-align', 'center')          
                     ]},
                    {'selector': 'tbody td',      
                     'props': [
                         #('background-color', 'white'),#'#C2C7CA'), 
                         ('color', 'black'),              
                         ('font-family', 'Montserrat, sans-serif'),
                         ('text-align', 'center'),
                         ('min-width', '60px'),          
                     ]}
                ])
                .map(highlight_rows)
                .set_properties(**{
                    'border-collapse': 'collapse', 
                     'border': '1px solid grey' # Ensure no gaps between cells
                })
                .hide(axis='index')  # Hide index
             )
(styled_df)

P1,P2,P3,P4,P5,P6
4030,4030,4030,4030,4030,4830


In [ ]:
# Function to apply color gradient based on allowed power
def color_gradient(val, allowed_val):
    """Apply color based on comparison between real and allowed power."""
    try:
        ratio = val / allowed_val
        # Return color based on the ratio
        if ratio < 1:
            return f'background-color: rgba(0, 255, 0, {1 - ratio})'  # Green, lower ratio, more transparent
        elif ratio == 1:
            return 'background-color: rgba(255, 255, 0, 0.5)'  # Yellow for exact match
        else:
            return 'background-color: rgba(255, 0, 0, {ratio - 1})'  # Red, exceeded power
    except (ZeroDivisionError, TypeError, IndexError):
        return 'background-color: rgba(255, 0, 0, 0.3)'  # Light red if division by zero or invalid data

# Function to create a DataFrame of styles
def apply_gradient_to_dataframe(df_real, df_allowed):
    """Create a DataFrame of style strings based on comparison with allowed values."""
    styled_df = df_real.copy()
    for col in df_allowed.columns:
        if col in df_real.columns:
            allowed_value = df_allowed.at[0, col]
            styled_df[col] = df_real[col].apply(lambda x: color_gradient(x, allowed_value))
    return styled_df

# Apply styles using the function
styled_df = (df_maximetros_filtered.style
                .format("{:.0f}", subset=numeric_cols)  # Format numeric columns with no decimals
                .set_table_styles([
                    {'selector': 'thead th',
                     'props': [
                         ('background-color', '#012e41'),  
                         ('font-family', 'Montserrat, sans-serif'),
                         ('border', '1px solid grey'),
                         ('color', 'white'),
                         ('text-align', 'center')
                     ]},
                    {'selector': 'tbody td',
                     'props': [
                         ('color', 'black'),
                         ('font-family', 'Montserrat, sans-serif'),
                         ('text-align', 'center'),
                         ('min-width', '60px'),
                     ]}
                ])
                .apply(lambda x: apply_gradient_to_dataframe(df_maximetros_filtered, df_potencia_filtered), axis=None)  # Apply the style function correctly
                .set_properties(**{
                    'border-collapse': 'collapse',
                    'border': '1px solid grey'  # Ensure no gaps between cells
                })
                .hide(axis='index')  # Hide index
             )

# Display the styled DataFrame
styled_df


KeyError: 0

In [ ]:
import dataframe_image as dfi

In [ ]:
coste_unidad = (df_invoices_filtered['Coste Energía'])/(df_energia_filtered['Total'])
coste_unidad

1896     79.604427
1898    106.291366
1900    113.508718
1902     98.207349
1904     98.795529
1906    135.236763
1908    144.707458
1882    140.212585
1885    160.882170
1888     91.772953
1891     69.313968
1894     73.621745
dtype: float64

In [ ]:
coste_prom = (coste_unidad * df_energia_filtered['Total']).sum() / df_energia_filtered['Total'].sum()
coste_prom

np.float64(108.49088493959857)